In [ ]:
from gitloc import get_commits
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (15,5)

# Specify a repo (or remove string parameter to visualise this repo)
commits = get_commits("../my-favorite-repo")

In [ ]:
import numpy as np

df = pd.DataFrame(commits).set_index("date")

df["net"] = df["adds"] - df["deletes"]
df["locs"] = df["net"].cumsum()

start_date = df.index.min()
end_date = df.index.max()
#start_date = "2022-06-01"
#end_date = "2023-01-01"

timespan = end_date - start_date

scale, date_format = "W-MON", "%Y-%m-%d"
if timespan >= pd.Timedelta(days=365):
    scale, date_format = "MS", "%Y-%m"
if timespan >= pd.Timedelta(days=1000):
    scale, date_format = "YS", "%Y"

df = df.loc[start_date:end_date]

df


In [ ]:
p = sns.lineplot(df["locs"], drawstyle='steps-post')
dates = list(pd.date_range(start=start_date, end=end_date, freq=scale).to_pydatetime())
xticklabels = [date.strftime(date_format) for date in dates]
plt.xticks(dates, xticklabels, rotation=45)
p.set_ylim(0, df["locs"].max() * 1.1)
p.grid()
p.set_ylabel("total lines of code")
p

In [ ]:
df_frequency = pd.DataFrame({"commits": [1] * len(df)}, index=df.index)
df_frequency = df_frequency.resample(scale).count()
df_frequency.index = df_frequency.index.strftime(date_format)
p = sns.barplot(x=df_frequency.index, y="commits", data=df_frequency)
plt.xticks(rotation=45)
p.grid()
p

In [ ]:
df_changes = df[['adds', 'deletes']]
df_changes["deletes"] = -df_changes["deletes"]
df_changes = df_changes.resample(scale).sum()
df_changes.index = df_changes.index.strftime(date_format)
p = sns.barplot(x=df_changes.index, y="adds", data=df_changes)
p = sns.barplot(x=df_changes.index, y="deletes", data=df_changes)
plt.xticks(rotation=45)
p.set_ylabel("lines changed")
p.grid()
p